# Reference
[2. add tools - tutorial before langgraph v1.0](https://langchain-ai.github.io/langgraph/tutorials/get-started/2-add-tools/)

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### 3. Define the tool

In [3]:
from langchain_tavily import TavilySearch
tool = TavilySearch(max_results=2)
tools = [tool]
tool.invoke("What's a 'node' in Langgraph?")

{'query': "What's a 'node' in Langgraph?",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://medium.com/@kbdhunga/beginners-guide-to-langgraph-understanding-state-nodes-and-edges-part-1-897e6114fa48',
   'title': "Beginner's Guide to LangGraph: Understanding State, Nodes, and ...",
   'content': 'Nodes: Nodes are the fundamental building blocks of a graph. Each node represents a specific function or operation that processes the current',
   'score': 0.8252664,
   'raw_content': None},
  {'url': 'https://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141',
   'title': "Introduction to LangGraph: A Beginner's Guide - Medium",
   'content': '*   **Stateful Graph:** LangGraph revolves around the concept of a stateful graph, where each node in the graph represents a step in your computation, and the graph maintains a state that is passed around and updated as the computation progresses. LangGraph supports conditional edges, a

### 4. Define the graph

add `bind_tools` on the LLM -> LLM knows the correct JSON format to use search engine

In [5]:
from langchain.chat_models import init_chat_model
llm = init_chat_model('openai:gpt-4.1')

In [6]:
from typing import Annotated

from typing import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages:Annotated[list, add_messages]
    
graph_builder = StateGraph(State)

llm_with_tools = llm.bind_tools(tools) # Modification: tell the LLM which tools it can call

def chatbot(state:State):
    return {'messages': [llm_with_tools.invoke(state['messages'])]}

graph_builder.add_node('chatbot', chatbot)

### 5. Create a function to run the tools

In [7]:
import json

from langchain_core.messages import ToolMessage

class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage"""
    
    def __init__(self, tools:list) -> None:
        self.tools_by_name = {tool.name: tool for tool in tools}
    
    def __call__(self, inputs:dict):
        if messages := inputs.get("messages", []):
            messages = messages[-1]
        else:
            raise ValueError("No message found in input")
        
        outputs = []
        for tool_call in messages.tool_calls:
            tool_result = self.tools_by_name[tool_call['name']].invoke(
                tool_call['args']
            ) # LLM이 사용할 tool을 추론하면 그 tool을 호출해서, 호출한 tool에 LLM이 추론한 args값을 넣어주기
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result),
                    name=tool_call['name'],
                    tool_call_id=tool_call['id']
                )
            )
        
        return {'messages': outputs}

tool_node = BasicToolNode(tools=[tool])
graph_builder.add_node('tools', tool_node)

### 6. Define the conditional_edges